In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [2]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

search_word = "BERT"
df = pd.read_csv('data/nlp-align_BERT.csv')

print(df)

In [4]:
def split_search_term_literal(row, search_word):
    # Splits on first instance ONLY
    if search_word in row['Text']:
        splitted = row['Text'].split(search_word, maxsplit=1)
        splitted.insert(1, search_word)
    else:
        splitted = [row['Text'],'','']
    return dict(zip(['text_split_0','text_split_1','text_split_2'],splitted))

# df = df.join()
df.apply(lambda row: split_search_term_literal(row, search_word), axis=1, result_type='expand')

,text_split_0,text_split_1,text_split_2
0,,BERT,: Pre-training of Deep Bidirectional Transform...
1,We introduce a new language representation mod...,BERT,", which stands for Bidirectional Encoder Repre..."
2,"Unlike recent language representation models,",BERT,is designed to pre-train deep bidirectional r...
3,"As a result, the pre-trained",BERT,model can be fine-tuned with just one additio...
4,,BERT,is conceptually simple and empirically powerf...
5,It obtains new state-of-the-art results on ele...,,
6,Ro,BERT,a: A Robustly Optimized BERT Pretraining Approach
7,Language model pretraining has led to signific...,,
8,"Training is computationally expensive, often d...",,
9,We present a replication study of,BERT,"pretraining (Devlin et al., 2019) that carefu..."


In [5]:
def split_search_term_whitespace(row, search_word):
    # Splits on first instance ONLY
    if search_word in row['Text']:
        tokens = row['Text'].split(' ')
        for i in range(len(tokens)):
            if search_word in tokens[i]:
                splitted=[' '.join(tokens[:i]), tokens[i], ' '.join(tokens[i+1:])]
                break
    else:
        splitted = [row['Text'],'','']
    return dict(zip(['text_split_0','text_split_1','text_split_2'],splitted))

df.apply(lambda row: split_search_term_whitespace(row, search_word), axis=1, result_type='expand')

,text_split_0,text_split_1,text_split_2
0,,BERT:,Pre-training of Deep Bidirectional Transformer...
1,We introduce a new language representation mod...,"BERT,",which stands for Bidirectional Encoder Represe...
2,"Unlike recent language representation models,",BERT,is designed to pre-train deep bidirectional re...
3,"As a result, the pre-trained",BERT,model can be fine-tuned with just one addition...
4,,BERT,is conceptually simple and empirically powerful.
5,It obtains new state-of-the-art results on ele...,,
6,,RoBERTa:,A Robustly Optimized BERT Pretraining Approach
7,Language model pretraining has led to signific...,,
8,"Training is computationally expensive, often d...",,
9,We present a replication study of,BERT,"pretraining (Devlin et al., 2019) that careful..."


In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
predictor.predict(
  document="The woman reading a newspaper sat on the bench with her dog."
)

[nltk_data] Downloading package punkt to /home/cephcyn/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/cephcyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
